# Inference for Hidden Markov Models

Probabilistic Modelling and Reasoning 2022-2023. University of Edinburgh.

In this note, we will study and implement inference algorithms for a hidden Markov model (HMM).

The notes in this tutorial are based on the PMR lecture slides, which you can find [here](https://www.inf.ed.ac.uk/teaching/courses/pmr/).

## Table of Content

* Inference basics
* Forward-backward
* Viterbi-backtracking
* Forward-filtering backward-sampling

# Inference basics

In slides on "Exact Inference for Hidden Markov Models", the following inference problems were introduced:
* Filtering (inferring the present): $p(h_t | v_{1:t})$
* Smoothing (infering the past):  $p(h_t | v_{1:T}), t < T$, with $T$ being the lenght of the observed sequence
* Most likely hidden path: $h^*_{1:T} = \text{argmax}_{h_{1:T}} p(h_{1:T} | v_{1:T})$
* Posterior Sampling: $h_{1:T} \sim p(h_{1:T} | v_{1:T})$

In this notebook, we will implement the following inference algorithms for a discrete-valued HMM model specified in terms of probability tables:
* The alpha-recursion and beta-recursion algorithms as `forward` and `backward` functions, respectively.
* The Viterbi and Viterbi-backtracking algorithms.
* The forward-filtering backward-sampling algorithm.

These inference operations will allow us to solve the previously mentioned inference problems.

To start, recall the simple language example from the previous tutorial, where sentences were generated by templates
* Hidden template: Subject -> Verb -> Object
* Observed sentence: Jack loves dogs

Suppose we just observe the sentence but do not know the template. The inference problems then translate to:

* Filtering and smoothing, i.e. having observed the sentence "Jack loves dogs", what is the probability of:
    * the first latent state  $p(h_1 | v_{1:3} = \text{"Jack loves dogs"})$
    * the second latent state $p(h_2 | v_{1:3} = \text{"Jack loves dogs"})$
    * the third latent state  $p(h_3 | v_{1:3} = \text{"Jack loves dogs"})$
* Partition function, the marginal probability $p(v_{1:3} = \text{"Jack loves dogs"})$.
* The most likely hidden path $h^*_{1:3}$:
    * Having observed the sentence "Jack loves dogs", what is the most probabe latent template $h^*_{1:3}$ that generates this sentence, e.g. is it [Subject -> Verb -> Object]?
* Other hidden paths (i.e. templates) that are likely to generate "Jack loves dogs" sentence. Are there other hidden paths than [Subject -> Verb -> Object] that could also generate "Jack loves dogs"?

## Working in the log domain

Before diving into the details of the inference algorithms, we present some background on numerics. In brief, computers are more accurate when computing "good numbers" (not too small or too large in magnitude, say 5, 10, 100) than "bad numbers" (too small in magnitude like $10^{-10}$ or too large like $10^{10}$, try adding these two numbers in python yourself and see the result, also see how computers [represent floating point numbers](https://en.wikipedia.org/wiki/Single-precision_floating-point_format#IEEE_754_single-precision_binary_floating-point_format:_binary32)). To retain sufficient numerical precision we would prefer that all our inference algorithms worked with "good numbers".

Directly working with the raw probabilities may result in very small numbers. Consider a simple Markov chain transitioning 10 steps with each step having a transition probability of 0.1. Then, the probability for the full chain is $0.1^{10}$, which is a very small number, in fact, too small for computers to operate on accurately.

So we need to find a smarter way to represent this number. We do so by transforming it into the log-probability domain (natural logarithm) and perform any subsequent operations in the log domain only, e.g.: 

$\log 0.1^{10} = 10 \log 0.1 = -23.0258 \ldots$

Notice that we can represent $-23.0258$ with significantly more precision than $0.1^{10}$ because it is not too small or too large.

In the following sections, we will work with the log-probabilities to implement the algorithms introduced in the lecture.

# Forward-backward algorithm for computing the partition function and performing filtering and smoothing

## Forward algorithm: implementation

We start with the forward algorithm that computes the alpha-recursion. The core recursion is just one line (see "Exact Inference for Hidden Markov Models" slides):

$\alpha_t(j) = \sum_i \alpha_{t-1}(i)\phi_t(i, j) $

$\phi_t(i, j) = p(h_t = j| h_{t - 1} = i)p(v_t | h_t = j)$

But we need to be more specific and ask the following questions before implementing the code:
* What is the input to the algorithm?
* What are the initial values? 
* What is the output of the algorithm?

Note that these are the core questions that you should **always ask before implementing any algorithms.**

Here are the answers to the above questions:

Inputs:

* Initial probability $\pi(i)$: suppose there are $N$ latent states in total, then $\pi$ is a vector of length $N$ where $\pi(i)$ denotes the probability of state $i$ being the initial state, i.e. $p(h_1 = i)$.
* Transition probability $P$: again, there are $N$ latent states in total, then $P$ is a $N \times N$ shaped matrix where $P(i, j)$ denotes the probability of the transition from state $i$ to state $j$, i.e. $p(h_{t+1} = j \mid h_{t}=i)$.
* Emission probability $E$: suppose there are $V$ observed states in total, then $E$ is a $V \times N$ shaped matrix where $E(i, j)$ denotes the probability of hidden state $j$ emitting the observation $i$, i.e. $p(v_t=i \mid h_t=j)$.
* Observed sequence $v$: this is a vector of length $T$, where $v_t \in \{1, 2, ..., V\}$.

**It is extremely important to be clear about the shape of these vectors and matrices.** This will often help prevent bugs early in the implementation.
 

Initial values:

* $\alpha_1(i) = p(h_1 = i)p(v_1 | h_1 = i)= \pi(i) E(v_1, i)$
* What is the shape of $\alpha_1$?
* Answer: $\alpha_1$ is a vector of length $N$ (think about why).

Output:
* $\alpha_{1:T}$: A matrix of shape $T \times N$
* $Z_t$: a real valued number, the marginal probability $p(v)$ of the given observation

We now implement the forward algorithm below. Compare it with equations provided above and the description in "Exact Inference for Hidden Markov Models", also think about the questions provided as comments in the code.

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from scipy.special import logsumexp

def forward(log_initial, log_transition, log_emission, v):
    """
    Args:
        initial: a vector of length N
        transition: a matrix of shape N * N
        emission: a matrix of shape V * N
        v: observed sequence, a vector of length T
        
    Returns:
        log_alpha: a matrix of shape T * N, log_alpha[t, n] denote the sum of all possible latent state sequences ending at step t, state n
        log_Z: a float number
    """
    T = len(v)
    N = log_emission.shape[1]
    
    log_alpha = np.zeros((T, N)) 
    log_alpha[0] = log_emission[v[0]] + log_initial # Q: why this is an addition, not a multiplication?
    
    for t in range(1, T):
        log_emission_t = log_emission[v[t]]
        log_alpha[t] = logsumexp(log_alpha[t - 1].reshape(N, 1) +
                                 log_transition + 
                                 log_emission_t.reshape(1, N),
                                 axis=0) # Q: what does the `logsumexp` function do here? 
                                         # Q: what does the function `reshape` do here? 

    log_Z = logsumexp(log_alpha[T - 1], axis=0)  # Q: what does the `logsumexp` function do here? 
    return log_alpha, log_Z

Here are answers to the in-line questions:

* Multiplications in the probability domain $p(v \mid h) \pi(h)$ become additions in the log-domain $\log \left( p(v \mid h) \pi(h) \right) = \log p(v \mid h) + \log \pi(h)$ (see e.g. [wiki](https://en.wikipedia.org/wiki/Logarithm#Product,_quotient,_power,_and_root)).
* The `log_alpha[t] = ...` lines correspond to the equation $\sum_i \alpha_{t-1}(i)\phi_t(i, j)$ computed in the log-domain:
  $$
  \log \sum_i \alpha_{t-1}(i)\phi_t(i, j) = \log \sum_i \exp\left( \log \alpha_{t-1}(i) + \log \phi_t(i, j)   \right)
  $$
  Hence the `logsumexp`, which computes the operation in a numerically more stable way. Here is a short [blog post](https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/) that explains the numerical stability advantage and implementation of `logsumexp` in more detail.
* The `reshape` operation on `log_alpha` makes sure that the sum operation in $\log \alpha_{t-1}(i) + \log \phi_t(i, j)$ is performed on the index $i$.
* The `reshape` operation on `log_emission_t` makes sure that the sum operation in $\log p(h_t = j \mid h_{t - 1} = i) + \log p(v_t \mid h_t = j)$ is performed on the index $j$.
* The `log_Z = ...` line corresponds to computing the partition function $\log p(v_{1:T}) = \log \sum_{h_T} \exp \log \alpha_T(h_T) = \log \sum_{h_T} \alpha_T(h_T)$.

## Forward algorithm: verifying the implementation 

How would you verify that the above implementation is correct? 

Perhaps the simplest way would be hand-calculating the probabilities on a simple example problem and see if the algorithm's output matches the hand-calculated values.

Firstly we define the parameters of the simple HMM: i.e. initial, transition, and emission probabilities.

In [2]:
log_initial = np.log(np.array([0.3, 0.7]))
log_transition = np.log(np.array([[0.2, 0.8], 
                                  [0.6, 0.4]]))
log_emission = np.log(np.array([[0.6, 0.7], 
                                [0.4, 0.3]]))

This is a simple HMM with only two hidden states and two types of observations. Now we enumerate all observations of length 2:

In [3]:
v0 = [0, 0]
v1 = [0, 1]
v2 = [1, 0]
v3 = [1, 1]

Then we call the forward algorithm to calculate the probabilities of these observations

In [4]:
log_Zs = []
for v in [v0, v1, v2, v3]:
    log_alpha, log_Z = forward(log_initial, log_transition, log_emission, v)
    log_Zs.append(log_Z)

And print out all the probabilities from the algorithm. Note that we exponentiate the log-probabilities back to the raw probability domain:

In [5]:
probs = [np.exp(log_Z) for log_Z in log_Zs]
print(probs)

[0.4359999999999999, 0.23399999999999996, 0.21600000000000003, 0.114]


Your task: calculate the probabilities for the observations (say calculate the last one [1, 1]) by hand, and verify the calculated probability matches the code output).

Answer: 

$$
\begin{align*}
p(v = [1,1]) &= \sum_{h_1, h_2} p(v_1=1 \mid h_1) p(h_1) p(h_2 \mid h_1) p(v_2=1 \mid h_2)\\
&= \left. p(v_1=1 \mid h_1) p(h_1) p(h_2 \mid h_1) p(v_2=1 \mid h_2) \right|_{h_1 = 0, h_2 = 0}\\
&\phantom{=} + \left. p(v_1=1 \mid h_1) p(h_1) p(h_2 \mid h_1) p(v_2=1 \mid h_2) \right|_{h_1 = 0, h_2 = 1}\\
&\phantom{=} + \left. p(v_1=1 \mid h_1) p(h_1) p(h_2 \mid h_1) p(v_2=1 \mid h_2) \right|_{h_1 = 1, h_2 = 1}\\
&\phantom{=} + \left. p(v_1=1 \mid h_1) p(h_1) p(h_2 \mid h_1) p(v_2=1 \mid h_2) \right|_{h_1 = 1, h_2 = 0}\\
&= 0.4*0.3*0.2*0.4\\
&\phantom{=} + 0.4*0.3*0.8*0.3\\
&\phantom{=} + 0.3*0.7*0.4*0.3\\
&\phantom{=} + 0.3*0.7*0.6*0.4\\
&= 0.114
\end{align*}
$$

Additionally, a good sanity check practice is to sum them over and see if the result equals to one (Q: why the results should equal to one?).

In [6]:
np.sum(probs)

0.9999999999999999

It's OK if the resulting number is not exactly 1.0. It happens due to the rounding errors when working with floating point numbers. 

Yet, if the error is more than ~$10^{-5}$, you should  check if there are bugs. 

## Forward algorithm: simple language model example

Let us now revisit the simple language model example from the previous tutorial. 

We first specify an HMM model with the same initial, transition, and emission probability tables as before.
Then we compute the probability of the sentences, e.g. $p(v = \text{"Jack loves dogs"})$, under the specified model.

In [7]:
id2state = {0: 'Subject', 1: 'Adjective', 2: 'Adverb', 3: 'Verb', 4: 'Object', 5: '<EOS>'}
state2id = {id2state[s]: s for s in id2state}
word2id = {'I': 0, 'He': 1, 'Jack': 2, 'Mary': 3, 'likes': 4, 'loves': 5, 'hates': 6, 'really': 7, 'extremely': 8, 'pretty': 9, 'cute': 10, 'adorable': 11, 'cats': 12, 'dogs': 13, '.': 14}
id2word = {word2id[w]: w for w in word2id}


log_initial_lang = np.log([0.7, 0.3, 0., 0., 0., 0.])
log_transition_lang = np.log(np.array([[0., 0., 0.3, 0.7, 0., 0.], 
                                       [0.4, 0.1, 0., 0., 0.5, 0.], 
                                       [0., 0.3, 0., 0.7, 0., 0.],
                                       [0., 0.3, 0.2, 0., 0.5, 0.],
                                       [0., 0., 0., 0., 0., 1.],
                                       [0., 0., 0., 0., 0., 1.],
                                      ]))
log_emission_lang = np.log(np.array([
    [0.2, 0.2, 0.2, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0.1, 0.1, 0.],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0.25, 0.25, 0, 0, 0.],
    [0, 0, 0, 0, 0, 0, 0, 0.25, 0.25, 0.5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0.3, 0.4, 0.3, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0.2, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0.3, 0.3, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.]
    ])).T

What are the probabilities of the following sentences under our model?
* "Jack loves dogs."
* "Mary likes adorable cats."
* "like dogs Jack."

In [8]:
v = ["Jack", "loves", "dogs", "."]
v_id = [word2id[word] for word in v]
log_alpha, log_Z = forward(log_initial_lang, log_transition_lang, log_emission_lang, v=v_id)
print(" ".join(v), f'p={np.exp(log_Z)}')

v = ["Mary", "likes", "adorable", "cats", "."]
v_id = [word2id[word] for word in v]
log_alpha, log_Z = forward(log_initial_lang, log_transition_lang, log_emission_lang, v=v_id)
print(" ".join(v), f'p={np.exp(log_Z)}')

v = ["likes", "dogs", "Jack", "."]
v_id = [word2id[word] for word in v]
log_alpha, log_Z = forward(log_initial_lang, log_transition_lang, log_emission_lang, v=v_id)
print(" ".join(v), f'p={np.exp(log_Z)}')

Jack loves dogs . p=0.005879999999999998
Mary likes adorable cats . p=0.0003307499999999996
likes dogs Jack . p=0.0


Note that our HMM has a 0.0 probability to generate an incorrect sentence "likes dogs Jack.", while the probabilities are nonzero for the grammatically correct sentences.

We can also use the forward algorithm to perform filtering: to infer the probability of a hidden state $h_t$ given the observed states $v_{1:t}$: $p(h_t \mid v_{1:t})$.

In [9]:
v = ["cute", "Jack", "loves", "dogs", "."]
v_id = [word2id[word] for word in v]

for t in range(1, len(v)+1):
    log_alpha, log_Z = forward(log_initial_lang, log_transition_lang, log_emission_lang, v=v_id[:t])
    p = {id2state[i]: round(p,2) for i, p in enumerate(np.exp(log_alpha[-1] - log_Z))}
    sentence = " ".join(v[:t])
    print(f'p(h_{t} | v[1:{t}] = "{sentence}")=\n {p}')

p(h_1 | v[1:1] = "cute")=
 {'Subject': 0.0, 'Adjective': 1.0, 'Adverb': 0.0, 'Verb': 0.0, 'Object': 0.0, '<EOS>': 0.0}
p(h_2 | v[1:2] = "cute Jack")=
 {'Subject': 0.44, 'Adjective': 0.0, 'Adverb': 0.0, 'Verb': 0.0, 'Object': 0.56, '<EOS>': 0.0}
p(h_3 | v[1:3] = "cute Jack loves")=
 {'Subject': 0.0, 'Adjective': 0.0, 'Adverb': 0.0, 'Verb': 1.0, 'Object': 0.0, '<EOS>': 0.0}
p(h_4 | v[1:4] = "cute Jack loves dogs")=
 {'Subject': 0.0, 'Adjective': 0.0, 'Adverb': 0.0, 'Verb': 0.0, 'Object': 1.0, '<EOS>': 0.0}
p(h_5 | v[1:5] = "cute Jack loves dogs .")=
 {'Subject': 0.0, 'Adjective': 0.0, 'Adverb': 0.0, 'Verb': 0.0, 'Object': 0.0, '<EOS>': 1.0}


Note the resulting probabilities for $p(h_2 | v_{1:2} = \text{"cute Jack"})$ are not certain whether the hidden state of $h_2$ is an object or a subject. Think about why this may be the case. We provide an answer in the smoothing section below.

## Backward algorithm: implementation

Compare the following implementation with slides on "Exact Inference for Hidden Markov Models", and with the implementation of the forward algorithm.

In [10]:
def backward(log_initial, log_transition, log_emission, v):
    """
    Args:
        initial: a vector of length N
        transition: a matrix of shape N * N
        emission: a matrix of shape V * N
        v: observed sequence, a vector of length t

    Returns:
        log_beta: a matrix of shape T * N
        log_Z: a float number
    """
    T = len(v)
    N = log_emission.shape[1]
    
    log_beta = np.zeros((T, N)) # Q: Why is it initialised to zero?
    
    for t in range(T - 2, -1, -1):
        log_emission_t = log_emission[v[t + 1]]
        log_beta[t] = logsumexp(log_beta[t + 1].reshape(1, N) + 
                                log_transition + 
                                log_emission_t.reshape(1, N), 
                                axis=1)
    log_Z = logsumexp(log_beta[0] + log_emission[v[0]] + log_initial) # Q: why is it equal to the log-likelihood? 

    return log_beta, log_Z

Answers:

* In the backward algorithm $\beta_T(h_s) = 1$, hence in the log domain $\log \beta_T(h_s) = 0$.
* Recall from slides that $\beta_s(h_{s}) = p(v_{s+1:T} \mid h_{s})$.  
  Hence, $p(v_{1:T}) = \sum_{h_1} \beta_1(h_1) p(v_1 \mid h_1) p(h_1) = \sum_{h_1} p(v_{2:T} \mid h_{1}) p(v_1 \mid h_1) p(h_1)$

NB: Be careful about the boundaries, i.e., the initial values `log_beta[0]` and the final output `log_Z` . 

Compare this with the above forward algorithm. What are the differences before and after the loop? What are the differences inside the loop?

## Backward algorithm: verifying the implementation

We firstly check the partition functions produced by the backward algorithm, which should be the same as the forward algorithm:

In [11]:
log_Zs = []
for v in [v0, v1, v2, v3]:
    log_beta, log_Z = backward(log_initial, log_transition, log_emission, v)
    log_Zs.append(log_Z)

In [12]:
probs = [np.exp(log_Z) for log_Z in log_Zs]
print(probs, np.sum(probs))

[0.4359999999999999, 0.234, 0.21599999999999991, 0.114] 0.9999999999999999


Note that the above numbers are the same as the output of the forward algorithm, hence giving us confidence that the implementation is correct.

## Combining forward and backward (alpha and beta) to get all marginals

Finally, we combine the alpha and beta variables to get the marginal probabilities $p(h_t|v_{1:T})$ for each step $t$. The equations are given in "Exact Inference for Hidden Markov Models":

$p(h_t | v_{1:T}) = \frac{1}{Z}\alpha(h_t)\beta(h_t)$

Again, we compute this in the log domain. 

In [13]:
v = [0, 1, 0, 1] # an example observation sequence. Try other sequences yourself.

log_alpha, log_Z = forward(log_initial, log_transition, log_emission, v)
log_beta, log_Z = backward(log_initial, log_transition, log_emission, v)
log_marginal = log_alpha + log_beta - log_Z

In [14]:
np.exp(log_marginal)

array([[0.23806971, 0.76193029],
       [0.58981233, 0.41018767],
       [0.31313673, 0.68686327],
       [0.53619303, 0.46380697]])

Note that the shape of output `log_marginal` is $T \times N$ where `log_marginal[t, i]` represents the marginal probability $\log p(h_t = i|v_{1:T})$. In the next tutorial we will use this marginal probability in the EM algorithm for learning HMM parameters from data.

We can also further confirm that the distribution corresponds to a conditional probability table by summing the rows and checking that they are equal to 1:

In [15]:
assert np.allclose(np.exp(log_marginal).sum(-1), 1.)

## Smoothing: simple language model example

We can now perform smoothing, i.e. compute $p(h_t | v_{1:T})$ for $t < T$, on the simple language model.

Let us compute the marginals $p(h_t | v_{1:T})$ for the sentence "cute Jack loves dogs." from the previous example. Compare the marginals with the probabilities $p(h_t | v_{1:t})$ that we obtained from filtering, why are they different?

In [16]:
v = ["cute", "Jack", "loves", "dogs", "."]
v_id = [word2id[word] for word in v]

log_alpha, log_Z = forward(log_initial_lang, log_transition_lang, log_emission_lang, v=v_id)
log_beta, log_Z = backward(log_initial_lang, log_transition_lang, log_emission_lang, v=v_id)

log_marginal = log_alpha + log_beta - log_Z
marginal = np.exp(log_marginal)

sentence = " ".join(v)
for t in range(1, len(v)+1):
    p = {id2state[i]: round(p,2) for i, p in enumerate(marginal[t-1])}
    print(f'p(h_{t} | v[1:T] = "{sentence}")=\n {p}')


p(h_1 | v[1:T] = "cute Jack loves dogs .")=
 {'Subject': 0.0, 'Adjective': 1.0, 'Adverb': 0.0, 'Verb': 0.0, 'Object': 0.0, '<EOS>': 0.0}
p(h_2 | v[1:T] = "cute Jack loves dogs .")=
 {'Subject': 1.0, 'Adjective': 0.0, 'Adverb': 0.0, 'Verb': 0.0, 'Object': 0.0, '<EOS>': 0.0}
p(h_3 | v[1:T] = "cute Jack loves dogs .")=
 {'Subject': 0.0, 'Adjective': 0.0, 'Adverb': 0.0, 'Verb': 1.0, 'Object': 0.0, '<EOS>': 0.0}
p(h_4 | v[1:T] = "cute Jack loves dogs .")=
 {'Subject': 0.0, 'Adjective': 0.0, 'Adverb': 0.0, 'Verb': 0.0, 'Object': 1.0, '<EOS>': 0.0}
p(h_5 | v[1:T] = "cute Jack loves dogs .")=
 {'Subject': 0.0, 'Adjective': 0.0, 'Adverb': 0.0, 'Verb': 0.0, 'Object': 0.0, '<EOS>': 1.0}


Answer:

The difference is in $p(h_2 \mid v_{1:T})$ (smoothing) and $p(h_2 \mid v_{1:2})$ (filtering). Since filtering does not use "future" observations it couldn't tell whether the word "Jack" was the object or the subject of the sentence. On the other hand, by incorporating "future" observations (i.e. the following words "... loves dogs."), smoothing can correctly infer that "Jack" is the subject of the sentence.

# Viterbi-backtracking for computing the most probable latent sequence

We now switch our attention to another important problem, given an observed sequence, what is the probability of the most probable latent sequence:

$\max_{h_{1:T}} p(h_{1:T} | v_{1:T})$

We can compute this using the Viterbi algorithm from Tutorial 5. 

The implementation of the Viterbi algorithm (in the non log-domain) resembles that of the forward algorithm: it replaces the `sum` in the forward algorithm to a `max`, as follows:

* Forward recursion: $\alpha_t(j) = \sum_i \alpha_{t-1}(i)\phi_t(i, j) $
* Viterbi recursion: $s_t(j) = \max_i s_{t-1}(i)\phi_t(i, j) $

This is due to the more general relationship between sum-product and max-product message passing. Here we change the intermediate variables from $\alpha$ to $s$ just for clarity. 

Following the max-sum algorithm (as used in Tutorial 5), we implement the Viterbi algorithm in the log-domain below:

In [17]:
def viterbi(log_initial, log_transition, log_emission, v):
    """
    Args:
        initial: a vector of length N
        transition: a matrix of shape N * N
        emission: a matrix of shape V * N
        v: observed sequence, a vector of length T
        
    Returns:
        max_log_p: a float number, maximum probability of the latent sequence
    """
    T = len(v)
    N = log_emission.shape[1]
    
    log_s = np.zeros((T, N)) 
    log_s[0] = log_emission[v[0]] + log_initial 
    
    for t in range(1, T):
        log_s[t] = np.max(log_s[t-1].reshape(N, 1) + 
                          log_transition + 
                          log_emission[v[t]].reshape(1, N), 
                          axis=0) # NOTE: change logsumexp to max here, compare this to Forward

    max_log_p_h = np.max(log_s[T - 1]) # NOTE: change the logsumexp to a max function here
    return max_log_p_h

However, the above implementation only gives us the $\max_{h_{1:T}} \log p(h_{1:T} | v_{1:T})$, i.e., the maximum of $\log p(h_{1:T} | v_{1:T})$. 

Most often we would also like to know the actual latent sequence that corresponds to the maximum probability, i.e. $\text{argmax}_{h_{1:T}} p(h_{1:T} | v_{1:T})$. We obtain this sequence by tracking back-pointers during the Viterbi algorithm and then backtracking after reaching the last step:

In [18]:
def viterbi_backtracking(log_initial, log_transition, log_emission, v):
    """
    Args:
        initial: a vector of length N
        transition: a matrix of shape N * N
        emission: a matrix of shape V * N
        v: observed sequence, a vector of length T
        
    Returns:
        max_log_p: a float number, maximum probability of the latent sequence
        max_h: max probability latent sequence, a vector of length T
    """
    T = len(v)
    N = log_emission.shape[1]
    
    log_s = np.zeros((T, N)) 
    max_ptr = np.zeros((T, N))
    log_s[0] = log_emission[v[0]] + log_initial 
    
    for t in range(1, T):
        log_phi_t = log_transition + log_emission[v[t]].reshape(1, N)
        log_s_phi_t = log_s[t-1].reshape(N, 1) + log_phi_t
        log_s[t] = np.max(log_s_phi_t, axis=0) 
        max_ptr[t] = np.argmax(log_s_phi_t, axis=0) # record the back pointer

    max_log_p_h = np.max(log_s[T - 1])
    
    # backtracking start
    max_h = np.zeros(T).astype(int)
    max_h[T - 1] = np.argmax(log_s[T - 1])
    for t in range(T - 2, -1, -1):
        max_h[t] = max_ptr[t + 1, max_h[t + 1]]
    return max_log_p_h, max_h

It is important to understand what happens during the backtracking procedure. Specifically, think about the following questions:
* What does `max_ptr[t][i]` mean? 
* What does the line `max_ptr[t + 1, max_h[t + 1]]` in the backtracking loop do? 

You may want to draw a figure to help yourself understand the backtracking process.

Answer:

* `max_ptr[t, i]` stores a look-up table that maps a state $h_{t+1}=i$ at timestep $t+1$ to the state index of $h_t$ that maximises $p(h_{1:t} | v_{1:t}, h_{t+1})$.
* In the backtracking loop it starts with the state $h_T$ at the final step $T$ that maximises $p(h_{1:T} | v_{1:T})$ and looks up the  index of the previous state $h_{t}$ that maximises $p(h_{1:t} | v_{1:t}, h_{t+1})$ from the look-up table at $t=T-1$. It continues until the first state in the Markov chain $t=1$ is reached.


## Verifying the Viterbi implementation

Let us run the Viterbi-backtracking code reusing the above simple HMM (2 latent states and 2 observed states)

In [19]:
log_initial = np.log(np.array([0.3, 0.7]))
log_transition = np.log(np.array([[0.2, 0.8], 
                                  [0.6, 0.4]]))
log_emission = np.log(np.array([[0.6, 0.7], 
                                [0.4, 0.3]]))

In [20]:
v = [0, 0]
_, log_Z = forward(log_initial, log_transition, log_emission, v)
max_log_p_h, max_h = viterbi_backtracking(log_initial, log_transition, log_emission, v)
print(np.exp(max_log_p_h), np.exp(max_log_p_h - log_Z), max_h)

0.17639999999999997 0.4045871559633028 [1 0]


This means that for the observed sequence v = [0, 0], the most probable latent sequence is h = [1, 0], and its corresponding probability is $p(h | v) \approx 0.4045$, and the joint probability $p(h, v) \approx 0.1763$.

Note that even though we performed the computations in the log-domain, i.e. we computed $h^*_{t:T} = \text{argmax}_{h_{1:T}} \log p(h_{1:T} | v_{1:T})$, the obtained latent sequence $h^*_{t:T}$ is the same as the one computed in the raw probability domain  $h^*_{t:T} = \text{argmax}_{h_{1:T}} p(h_{1:T} | v_{1:T})$, due to the $\log$ being a strictly increasing function.

You can verify that the implementation is correct by calculating the probability for all latent sequences (i.e., calculate the probabilities for $h = $[0, 0], [0, 1], [1, 0] or [1, 1]) by hand, and comparing the hand-calculated probability to the code output.

Note that the posterior probability $p(h | v)$ above is obtained by the Bayes rule: 

$p(h | v) = \frac{p(h, v)}{p(v)}$

where the probability of the observation $p(v)$ (the denominator) is obtained using the forward algorithm.

## Viterbi-backtracking: simple language model example

We now illustrate the Viterbi-backtracking on the simple language model to infer the most likely template of a sentence.

Let us consider the sentence "Mary likes pretty cute dogs". Note that we excluded puctuation in this example such that the final word may be either a subject or an object. (In our grammar only an object can directly transition into the \<EOS\> state, hence it immediately gives away that the previous word is an object).

In [21]:
v = ["Mary", "likes", "pretty", "cute", "dogs"]
v_id = [word2id[word] for word in v]

_, max_h = viterbi_backtracking(log_initial_lang, log_transition_lang, log_emission_lang, v=v_id)
print(' '.join([id2state[h] for h in max_h]))

Subject Verb Adverb Adjective Object


Hence, the most likely hidden path under the given HMM model is [Subject Verb Adverb Adjective Object]. We will later compare this with other possible hidden paths by sampling.

# Forward-filtering backward-sampling

We now consider the sampling of latent states $h$ given an observation $v$. Firstly, it is important to differentiate sampling from Viterbi-backtracking:
* Viterbi-backtracking: $h_{1:t}^* = \text{argmax } p(h_{1:T} | v_{1:t})$
    * There is only one $h_{1:T}^*$ that has the largest probability (under the assumption of a unique global maximum).
* Sampling: $h_{1:t} \sim p(h_{1:T} | v_{1:t})$
    * In sampling we attempt to generate several latent sequences $h_{1:T}$, not just the most probable.
    * Note that the sequence with $h_{1:T}^*$ still has the largest probability to get sampled.


It is also helpful to view sampling as a random function (as opposed to a deterministic function): 
* Viterbi-backtracking is a deterministic function since it always gives the same output.
* Sampling is a random function since each time it may give us different outputs.

Sampling from the posterior $p(h_{1:T} | v_{1:T})$ can be done with backward-sampling:

$$
\begin{align}
    h_T &\sim p(h_T | v_{1:T}) \\ 
    h_{t-1} &\sim p(h_{t-1} | h_t, v_{1:T}) 
\end{align}
$$

The problem is to calculate $p(h_{t-1} | h_t, v_{1:T})$, which is: 

$$
\begin{align}
p(h_{t - 1}| h_t, v_{1:T}) &= \frac{p(h_{t - 1}, h_t | v_{1:T})}{p(h_t | v_{1:T})} \\ 
    &= \frac{\alpha(h_{t-1}) p(h_t | h_{t-1})p(v_t | h_t) \beta(h_t) \frac{1}{Z}}{\alpha(h_t)\beta(h_t) \frac{1}{Z}} \\ 
    &= \frac{\alpha(h_{t-1})  p(h_t | h_{t-1}) p(v_t | h_t)}{\alpha(h_t)}
\end{align}
$$ 
where the first line is obtained by the Bayes' rule, the second line plugs in the definitions of $p(h_{t - 1}, h_t | v_{1:T})$ and $p(h_t | v_{1:T})$, and the final line is obtained by eliminating $\beta(h_t)$ and $Z$. (See Tutorial 5 for details.)

Since the probabilities above are expressed as a function of the transition, emission, and the alpha variables, we will use the forward algorithm to obtain them. This is why the sampling algorithm is called Forward-Filtering Backward-Sampling: because it calls the forward algorithm first and then samples in a backward fashion.

In [22]:
def sampling(log_initial, log_transition, log_emission, v):
    """
    Args:
        initial: a vector of length N
        transition: a matrix of shape N * N
        emission: a matrix of shape V * N
        v: observed sequence, a vector of length T
        
    Returns:
        h: sampled sequence, a vector of length T
    """
    log_alpha, log_Z = forward(log_initial, log_transition, log_emission, v)
    
    T = len(v)
    N = log_emission.shape[1]
    h = np.zeros(T).astype(int)
    p_h = np.zeros(T)
    
    p = np.exp(log_alpha[T - 1] - log_Z)
    h[T - 1] = np.random.choice(N, p=p)
    p_h[T - 1] = p[h[T - 1]]
    for t in range(T - 2, -1, -1):
        log_p = log_alpha[t].reshape(N, 1) +\
            log_transition +\
            log_emission[v[t + 1]].reshape(1, N) -\
            log_alpha[t + 1].reshape(1, N) # Q: why p is calculated this way?
        p = np.exp(log_p[:, h[t + 1]])
        h[t] = np.random.choice(N, p=p)
        p_h[t] = p[h[t]]
    return p_h, h

Answer:

The computation of the equations above are performed in the log domain, hence
$$
\begin{align*}
\log p(h_{t - 1}| h_t, v_{1:T}) &= \log \left(\frac{\alpha(h_{t-1}) p(h_t | h_{t-1}) p(v_t | h_t)}{\alpha(h_t)} \right)\\
&= \log \alpha(h_{t-1}) + \log p(h_t | h_{t-1}) + \log p(v_t | h_t) - \log \alpha(h_t)
\end{align*}
$$


## Forward-filtering backward-sampling: verifying the implementation

Again, we use the previous simple example HMM

In [23]:
log_initial = np.log(np.array([0.3, 0.7]))
log_transition = np.log(np.array([[0.2, 0.8], 
                                  [0.6, 0.4]]))
log_emission = np.log(np.array([[0.6, 0.7], 
                                [0.4, 0.3]]))

Then we call the sampling function to see its output. Try running it multiple times to see that the outputs can be different.

In [24]:
v = [0, 0]
p_h, h = sampling(log_initial, log_transition, log_emission, v)
print(np.prod(p_h), h)

0.3146788990825688 [1 1]


Finally, we verify our implementation by the law of large numbers: if we call the sampling function many times, then the frequency of each sampled latent sequence will be approximately equal to their probability

In [25]:
h_freq = {'00': 0, '01': 0, '10': 0, '11': 0}
N = 10000
for _ in range(N):
    p_h, h = sampling(log_initial, log_transition, log_emission, v)
    h_freq[f'{h[0]}{h[1]}'] += 1

for h in h_freq:
    print('latent %s freq %.4f' % (h, h_freq[h] / N))

latent 00 freq 0.0487
latent 01 freq 0.2310
latent 10 freq 0.4036
latent 11 freq 0.3167


Note that the frequency of the latent sequence [1, 0] returned from the simulation is close to 0.4045, i.e., approximately same as the true probability returned by the Viterbi algorithm. You may increase the number `N=100000`, which should return an even closer result.

## Forward-filtering backward-sampling: simple language model example

Let us now consider sampling the hidden paths (sentence templates) under our simple language HMM model, i.e. $h_{1:T} \sim p(h_{1:T} \mid v_{1:T})$. We use the same sentence as in the Viterbi-backtracking example "Mary likes pretty cute dogs". Note that running the below code generates different sentence templates, including the one obtained in the Viterbi-backtracking example.

In [26]:
v = ["Mary", "likes", "pretty", "cute", "dogs"]
v_id = [word2id[word] for word in v]

p_h, h = sampling(log_initial_lang, log_transition_lang, log_emission_lang, v=v_id)
print(' '.join([id2state[h] for h in h]))
print(np.prod(p_h))

Subject Verb Adjective Adjective Object
0.263157894736842


If you run the above code cell multiple times you may generate a template [Subject Verb Adjective Adjective Object], which is the correct template for this sentence. Note that it does not correspond to the result from Viterbi-backtracking example, which is due to our choice of the HMM model parameters, which make the template [Subject Verb Adverb Adjective Object] more likely for the sentence.